In [75]:
import numpy as np
import math
import scipy.stats
import matplotlib.pyplot as plt

In [76]:
def readExcelSheet1(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values

In [77]:
def readExcelRange(excelfile,sheetname="Sheet1",startrow=1,endrow=1,startcol=1,endcol=1):
    from pandas import read_excel
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]

In [78]:
def readExcel(excelfile,**args):
    if args:
        data=readExcelRange(excelfile,**args)
    else:
        data=readExcelSheet1(excelfile)
    if data.shape==(1,1):
        return data[0,0]
    elif (data.shape)[0]==1:
        return data[0]
    else:
        return data

In [79]:
def writeExcelData(x,excelfile,sheetname,startrow,startcol):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()

In [80]:
def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names

In [81]:
def linearRegressionTraining(X,T): #X is w/o x0. T is from 0 to m for m class labels
    X0 = np.full((len(X),1), 1, dtype=np.float)
    XC = np.hstack((X0,X))
    #Calculate the pseudo-inverse Matrix of X, by using numpy
    X_d = np.linalg.pinv(XC)
    tmax = np.amax(T);
    tmin = np.amin(T);
    if(tmax-tmin>2): #non binary multiclass
        TM= np.full((len(T),tmax-tmin+1), -1, dtype=np.float)
        for i,t in enumerate(T):
            TM[i][t]=1.0
    else:
        TM=T
    #Calculate the vector of weights w as multiplication of X_d and y
    w = np.dot(X_d,TM)
    return w

In [82]:
def applyLinearClassifier(X,W,th,num): #th is threshold. num is # of class labels
    X0 = np.full((len(X),1), 1, dtype=np.float)
    XC = np.hstack((X0,X))
    TP = np.dot(XC,W)
    TH = np.full((len(TP),), th, dtype=np.float)
    resultlabel = np.full((len(TP),), 0, dtype=np.int)
    if(num==2):        
        indicesp=TP>TH;
        indicesn=TP<TH;
        resultlabel[indicesp]=1;
        resultlabel[indicesn]=-1;
        #countp= np.sum(indicesp)
        #countn= np.sum(indicesn)
        #count = np.array([countn,countp])
        print(resultlabel[1:5],resultlabel.shape)
    else:
        resultlabel = np.argmax(TP, axis=1) #index of maximum on horizental axis
        #count = np.full((num,),0,dtype=np.int)
        #for i in range(num):
            #count[i]= (resultlabel==i).sum()
        print(resultlabel[1:5],resultlabel.shape)
    return resultlabel

In [83]:
def computeConfusionMatrix(resultlabel,T,num): #T is ground truth labels. num is # of class labels
    matrix = np.full((num,num), 0, dtype=np.int)
    for truth in range(num):
        for predict in range(num):
            count = np.full((len(resultlabel),), 0, dtype=np.int)
            count[((resultlabel==predict)&(T==truth))]=1
            matrix[truth][predict]=count.sum()
            
    ppv_denom = np.sum(matrix,axis=0) #it's a vector that sums each column
    ppv_top = np.full((num,), 0, dtype=np.int)
    for i in range(num):
        ppv_top[i]= matrix[i][i]
    ppv = ppv_top*100.0/ppv_denom; #a vector for all classes
    ppvmax_ind = np.argmax(ppv)
    ppvmin_ind = np.argmin(ppv)

    return matrix,ppv,ppvmax_ind,ppvmin_ind

In [84]:
excelfile=r"/Users/gholami/Documents/ucsc machine learning/assignment 4/Assignment_4_Data_and_Template.xlsx";
#excelfile_c=r"/Users/gholami/Documents/ucsc machine learning/assignment 4/Assignment_4_Data_and_Template copy.xlsx";

In [85]:
sheets=getSheetNames(excelfile);sheets

[u'Training Data', u'Classifiers', u'To be classified', u'Performance']

In [ ]:
data=readExcel(excelfile)
trainpct=0.75;
permuted_indices = np.random.permutation(len(data)); #divide data into two subset train and test
train = int(trainpct*len(data))
training_i = permuted_indices[:train]
test_i = permuted_indices[train:]

X_full = np.array(data[:,:15],dtype=float);
T_full=data[:,15]; #binary class labels
M_full=data[:,16]; #multi class labels

X_test = X_full[test_i]
T_test = T_full[test_i]
M_test = M_full[test_i]

X = X_full[training_i]
T = T_full[training_i]
M = M_full[training_i]

print (data.shape, X.shape,training_i.shape,test_i.shape )

count_b = countLabel(T_test,2)
count_b[0]=len(T_test)-count_b[1]

count_m = countLabel(M_test,6)
print(count_b,count_b.shape)
print(count_m,count_m.shape)

In [86]:
print (X_test)

[[ 154.  213.  201. ...,    0.    1.    0.]
 [ 279.   91.  176. ...,    1.    0.    0.]
 [ 145.  223.  178. ...,    1.    0.    0.]
 ..., 
 [ 219.  224.  214. ...,    0.    1.    0.]
 [ 295.  241.  291. ...,    0.    1.    0.]
 [ 208.  199.  132. ...,    1.    0.    0.]]


In [87]:
print (M_test)

[2 5 5 ..., 0 2 3]


In [88]:
w = linearRegressionTraining(X,T)
wM = linearRegressionTraining(X,M)

In [89]:
writeExcelData(w,excelfile,'Classifiers',5,1)
writeExcelData(wM,excelfile,'Classifiers',5,5)

In [90]:
queries=(readExcel(excelfile,
                  sheetname='To be classified',
                  startrow=5,
                  endrow=54,
                  startcol=1,
                  endcol=15)).astype(float);

In [91]:
resultlabel_qb=applyLinearClassifier(queries,w,0,2)
resultlabel_qm=applyLinearClassifier(queries,wM,0,6)
print (resultlabel_qb[0:10],resultlabel_qm[0:10], resultlabel_qm.shape)

(array([-1,  1, -1, -1]), (50,))
(array([4, 0, 4, 4]), (50,))
(array([-1, -1,  1, -1, -1, -1,  1, -1, -1, -1]), array([1, 4, 0, 4, 4, 5, 0, 4, 5, 1]), (50,))


In [92]:
writeExcelData(resultlabel_qb,excelfile,'To be classified',5,16)
writeExcelData(resultlabel_qm,excelfile,'To be classified',5,17)

In [93]:
resultlabel_tb = applyLinearClassifier(X_test,w,0,2)
TP = np.full(np.alen(T_test),0, dtype=int);
TP[((resultlabel_tb==1)&(T_test==1))]=1
TP_num=TP.sum() #true positive

TN = np.full(np.alen(T_test),0, dtype=int);
TN[((resultlabel_tb==-1)&(T_test==-1))]=1
TN_num=TN.sum()  #true negative

FN = np.full(np.alen(T_test),0, dtype=int);
FN[((resultlabel_tb==-1)&(T_test==1))]=1
FN_num=FN.sum()  #false negative

FP = np.full(np.alen(T_test),0, dtype=int);
FP[((resultlabel_tb==1)&(T_test==-1))]=1
FP_num=FP.sum()  #false positive

print (TP_num,TN_num,FN_num,FP_num,TP_num+TN_num+FN_num+FP_num)

(array([-1, -1, -1,  1]), (1650,))
(526, 825, 280, 19, 1650)


In [94]:
writeExcelData(np.reshape(TN_num,(1,1)),excelfile,'Performance',10,3)
writeExcelData(np.reshape(FP_num,(1,1)),excelfile,'Performance',10,4)
writeExcelData(np.reshape(FN_num,(1,1)),excelfile,'Performance',11,3)
writeExcelData(np.reshape(TP_num,(1,1)),excelfile,'Performance',11,4)
accB=(TP_num+TN_num)*1.0/(TP_num+TN_num+FN_num+FP_num);
sensB=TP_num*1.0/(TP_num+FN_num);
specB=TN_num*1.0/(TN_num+FP_num);
ppvB=TP_num*1.0/(TP_num+FP_num);
print accB,sensB,specB,ppvB
writeExcelData(np.reshape(accB,(1,1)),excelfile,'Performance',8,7)
writeExcelData(np.reshape(sensB,(1,1)),excelfile,'Performance',9,7)
writeExcelData(np.reshape(specB,(1,1)),excelfile,'Performance',10,7)
writeExcelData(np.reshape(ppvB,(1,1)),excelfile,'Performance',11,7)

0.818787878788 0.652605459057 0.977488151659 0.965137614679


In [95]:
resultlabel_tm = applyLinearClassifier(X_test,wM,0,6)

(array([5, 1, 4, 0]), (1650,))


In [96]:
matrix,ppv,high,low=computeConfusionMatrix(resultlabel_tm,M_test,6)
print(matrix,ppv,high,low)

(array([[159,   1, 112,   0,   0,  11],
       [  0, 217,   0,  22,   0,  30],
       [ 39,   0, 210,   0,   0,   5],
       [  6, 156,   1,  21,   0, 110],
       [  0,   3,   0,   0, 264,   2],
       [ 10,  88,   0,   5,   1, 177]]), array([ 74.29906542,  46.66666667,  65.01547988,  43.75      ,
        99.62264151,  52.8358209 ]), 4, 3)


In [97]:
writeExcelData(matrix,excelfile,'Performance',19,3)
writeExcelData(np.reshape(ppv[high],(1,1)),excelfile,'Performance',20,12)
writeExcelData(np.reshape(ppv[low],(1,1)),excelfile,'Performance',21,12)
writeExcelData(np.reshape(high,(1,1)),excelfile,'Performance',20,13)
writeExcelData(np.reshape(low,(1,1)),excelfile,'Performance',21,13)

In [2330]:
#[np.sum(H) for H in [HF, HFrec, HM, HMrec]]